In [2]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import catboost as cb
import re
import ast 
import numpy as np
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score
from sklearn.svm import SVC

2024-05-14 03:08:40.220381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
eth_ohlc = pd.read_csv('../data/transformed/ohlc/ethereum.csv')
eth_ohlc = eth_ohlc[:-1]
eth_ohlc

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_percent_increase_to_avg_high_low,tmw_0_0_percent_increase_binary,...,last_100_day_70th_to_80th_pct_spread_count,last_100_day_80th_to_90th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_70th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count,last_100_day_95th_pct_spread_count
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.033594,1.0,...,18.0,9.0,18.0,94.0,86.0,79.0,73.0,55.0,46.0,28.0
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,0.097955,1.0,...,18.0,9.0,18.0,94.0,86.0,78.0,72.0,54.0,45.0,27.0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,-0.009901,0.0,...,18.0,10.0,18.0,94.0,86.0,78.0,72.0,54.0,44.0,26.0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,-0.007323,0.0,...,18.0,10.0,17.0,94.0,86.0,78.0,71.0,53.0,43.0,26.0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,-0.015908,0.0,...,18.0,10.0,17.0,94.0,86.0,78.0,70.0,52.0,42.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,2024-05-07,3064.0000,3127.0000,3010.0000,3010.0000,1.030000e+10,3356192.0,3.756000e+11,-0.001495,0.0,...,8.0,4.0,3.0,47.0,33.0,25.0,15.0,7.0,3.0,0.0
3078,2024-05-08,3013.0000,3036.0000,2944.0000,2975.0000,1.020000e+10,3400618.0,3.675000e+11,0.023697,1.0,...,8.0,4.0,3.0,47.0,33.0,25.0,15.0,7.0,3.0,0.0
3079,2024-05-09,2977.0000,3055.0000,2958.0000,3036.0000,9.600000e+09,3190231.0,3.670000e+11,-0.018445,0.0,...,8.0,4.0,3.0,47.0,33.0,25.0,15.0,7.0,3.0,0.0
3080,2024-05-10,3035.0000,3050.0000,2889.0000,2910.0000,1.060000e+10,3557434.0,3.650000e+11,0.004467,1.0,...,8.0,4.0,3.0,48.0,34.0,25.0,15.0,7.0,3.0,0.0


In [4]:
column_names = eth_ohlc.columns.tolist()
print(column_names)

['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap', 'tmw_percent_increase_to_avg_high_low', 'tmw_0_0_percent_increase_binary', 'tmw_0_25_percent_increase_binary', 'tmw_0_5_percent_increase_binary', 'tmw_0_75_percent_increase_binary', 'tmw_1_0_percent_increase_binary', 'tmw_1_25_percent_increase_binary', 'tmw_1_5_percent_increase_binary', 'tmw_1_75_percent_increase_binary', 'tmw_2_0_percent_increase_binary', 'tmw_2_25_percent_increase_binary', 'tmw_2_5_percent_increase_binary', 'tmw_2_75_percent_increase_binary', 'tmw_3_0_percent_increase_binary', 'tmw_0_25_percent_decrease_binary', 'tmw_0_5_percent_decrease_binary', 'tmw_0_75_percent_decrease_binary', 'tmw_1_0_percent_decrease_binary', 'perc_from_low_to_high', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'sma_2', 'rsi_2', 'volume_sma_2', 'volume_rsi_2', 'sp

## Test each set of predictors generated with LASSO sliding window method in 03_ohlc_variable_analysis_and_selection.ipynb, and choose the 'best' set of predictors

In [5]:
predictor_sets = []

with open('selected_variables.txt', 'r') as file:
    for line in file:
        line = line.strip()
        components = line.split(', ')
        proportion = float(components[0])
        count = int(components[1])
        variables_list_str = ', '.join(components[2:])
        variables_list = ast.literal_eval(variables_list_str)
        predictor_sets.append((proportion, count, variables_list))
print(predictor_sets[0])

(0.0, 211, ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_50th_to_60th_pct_price_change_count', 'last_2_day_80th_to_90th_pct_price_change_count', 'last_2_day_60th_pct_price_change_count', 'last_2_day_70th_pct_price_change_count', 'last_2_day_50th_to_60th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_volume_change_count', 'last_2_day_70th_to_80th_pct_volume_change_count', 'last_2_day_40th_pct_volume_change_count', 'last_2_day_60th_pct_volume_change_count', 'last_2_day_70th_pct_volume_change_count', 'last_2_day_80th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_70th_to_80th_pct_spread_count', 'last_2_day_80th_to_90th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'last_2_day_60th_pct_spread_count', 'last_2_day_70th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 'spread_rsi_5', 'last_5_day_40th_to_50th_pct_price_change_count', 'last_5_day_50th_to_60th_pct_price_change_count', 'last_5_day_60th_to_70th_pct_price_change_count', 'last_5

### This is the backtesting function. Its like cross-validation, but for time series data. For a particular model, it returns a dataframe consisting of 'target' (what the actual value of the respone variable was) and 'probability', the predicted value of our model which signifies the proabality of a response of 1.

In [6]:
def get_backtested_predictions_df(data, predictors, model, response_var, start=1000, step=100):
    
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var])
        probability = model.predict_proba(test[predictors])[:,1]
        proba_series = pd.Series(probability, index=test.index, name="probability")
        combined = pd.concat([test[response_var], test['tmw_percent_increase_to_avg_high_low'], proba_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+100)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)
    
    return data_with_predictions

### arbitrarily use catboost to select the best predictor model

In [31]:
model_catboost = cb.CatBoostClassifier(iterations=300, learning_rate=.01, depth=6, verbose=False, random_seed=1)
# model_randomforest = RandomForestClassifier(n_estimators=250, min_samples_split=200, random_state=1)

In [27]:
list_of_increase_dfs_catboost = []
for proportion, num_predictors, predictor_list in predictor_sets:
    df = get_backtested_predictions_df(eth_ohlc, predictor_list, model_catboost, "tmw_1_0_percent_increase_binary")
    print(f'got predictions for model with predictor LASSO selection threshold {proportion}, num predictor = {num_predictors}')
    list_of_increase_dfs_catboost.append((df, proportion, num_predictors))

got predictions for model with predictor LASSO selection threshold 0.0, num predictor = 211
got predictions for model with predictor LASSO selection threshold 0.0127, num predictor = 197
got predictions for model with predictor LASSO selection threshold 0.0253, num predictor = 185
got predictions for model with predictor LASSO selection threshold 0.038, num predictor = 179
got predictions for model with predictor LASSO selection threshold 0.0506, num predictor = 171
got predictions for model with predictor LASSO selection threshold 0.0633, num predictor = 162
got predictions for model with predictor LASSO selection threshold 0.0759, num predictor = 153
got predictions for model with predictor LASSO selection threshold 0.0886, num predictor = 146
got predictions for model with predictor LASSO selection threshold 0.1013, num predictor = 141
got predictions for model with predictor LASSO selection threshold 0.1139, num predictor = 136
got predictions for model with predictor LASSO selecti

### Plot precision score versus the different predictor sets

In [33]:
def plot_ppv_by_threshold(list_of_dfs, use_over_0_percent_target, min_true_count, threshold_start, threshold_end, avg_ppv_greater_than_quantile, quantile_greater):
    # Initialize a Plotly figure
    fig = go.Figure()

    # Prepare to collect all traces data outside the loop
    traces_data = []

    for jawn in list_of_dfs:
        df = jawn[0]
        lasso_threshold = jawn[1]
        num_predictors = jawn[2]

        filtered_thresholds = []
        filtered_ppvs = []
        filtered_hover_texts = []  # List to store hover text information only for true_count >= min_true_count

        thresholds = [x * .25 for x in range(int(threshold_start / .25), int(threshold_end / .25) + 1)]
        for threshold in thresholds:
            col_name = f'pred_{threshold}'
            df[col_name] = df['probability'] >= (threshold / 100)
            
            if use_over_0_percent_target:
                cm = confusion_matrix(df['tmw_0_0_percent_increase_binary'], df[col_name])
            else:
                response_col_name = df.columns[0]
                cm = confusion_matrix(df[response_col_name], df[col_name])

            tn, fp, fn, tp = cm.ravel()
            ppv = tp / (tp + fp) if (tp + fp) > 0 else 0

            value_counts = df[col_name].value_counts().to_dict()
            false_count = value_counts.get(False, 0)
            true_count = value_counts.get(True, 0)

            if true_count >= min_true_count:
                filtered_thresholds.append(threshold)
                filtered_ppvs.append(ppv)
                hover_text = f"{num_predictors} Predictors (selection threshold={lasso_threshold})<br>Threshold: {threshold}<br>PPV: {ppv:.4f}<br>0: {false_count}<br>1: {true_count}"
                filtered_hover_texts.append(hover_text)

        # Calculate the average PPV for the current series
        average_ppv = np.mean(filtered_ppvs) if filtered_ppvs else 0

        # Store trace data in a list for adding outside the loop
        trace_data = {
            "x": filtered_thresholds,
            "y": filtered_ppvs,
            "text": filtered_hover_texts,
            "name": f'{num_predictors} Predictors (selection threshold={lasso_threshold})',
            "average_ppv": average_ppv
        }
        traces_data.append(trace_data)

    # Calculate the quantile value for the average PPV across all traces
    avg_ppv_quantile_value = np.quantile([data['average_ppv'] for data in traces_data], avg_ppv_greater_than_quantile)

    # Add traces that meet the quantile filter
    for data in traces_data:
        if quantile_greater:
            if data['average_ppv'] >= avg_ppv_quantile_value:
                fig.add_trace(go.Scatter(
                    x=data['x'],
                    y=data['y'],
                    mode='lines+markers',
                    name=data['name'],
                    text=data['text'],
                    hoverinfo="text"
                ))
        else: 
            if data['average_ppv'] <= avg_ppv_quantile_value:
                fig.add_trace(go.Scatter(
                    x=data['x'],
                    y=data['y'],
                    mode='lines+markers',
                    name=data['name'],
                    text=data['text'],
                    hoverinfo="text"
                ))

    fig.update_layout(
        title=f'Model Performance With Different Sets of Predictors (Filtered by sum(prediction=1) >= {min_true_count})',
        xaxis_title='Probability Threshold (%)',
        yaxis_title='Positive Predictive Value (PPV, Precision)',
        legend_title="Predictors",
        hovermode="closest"
    )

    fig.show()


In [39]:
plot_ppv_by_threshold(list_of_dfs=list_of_increase_dfs_catboost, 
                      use_over_0_percent_target=False, 
                      min_true_count=300, 
                      threshold_start=50, 
                      threshold_end=75, 
                      avg_ppv_greater_than_quantile=.90,
                      quantile_greater=True)

##### We'll go with the model with 96 predictors. This is not robust and is probably bullshit decision, but it's fine for this project since it is a a sufficient number of predicotrs to work with when optimizing different models below

## Now let's try several different models and hyperparameter combinations, and find the best one.

### Naive Model: If we had no model, and just bought Ethereum every single day, we would achieve a 1.0% return the following proportion of the time

In [33]:
print(eth_ohlc['tmw_1_0_percent_increase_binary'].value_counts(normalize=True))
print(eth_ohlc['tmw_1_0_percent_increase_binary'].value_counts())

tmw_1_0_percent_increase_binary
0.0    0.518131
1.0    0.481869
Name: proportion, dtype: float64
tmw_1_0_percent_increase_binary
0.0    1586
1.0    1475
Name: count, dtype: int64

 tmw_0_0_percent_increase_binary
1.0    0.656321
0.0    0.343679
Name: proportion, dtype: float64
tmw_0_0_percent_increase_binary
1.0    2009
0.0    1052
Name: count, dtype: int64


##### 1.0% returns occur 0.4819 proportion of the time, to random chance alone
##### < 0.0% returns occur 0.656321 proportion of the time, to random chance alone

### Model 1: Neural Network

In [90]:
def get_backtested_predictions_df_neural(data, predictors, model, response_var, start=1000, step=100):
    
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var], epochs=10, batch_size=32)
        probability = model.predict(test[predictors]).flatten()  # Using .flatten() to convert 2D array to 1D
        proba_series = pd.Series(probability, index=test.index, name="probability")
        combined = pd.concat([test[response_var], test['tmw_percent_increase_to_avg_high_low'], proba_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+100)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)
    
    return data_with_predictions

In [86]:
predictors_96 = ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_80th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 'spread_rsi_5', 'last_5_day_60th_pct_price_change_count', 'last_5_day_70th_pct_price_change_count', 'rsi_10', 'volume_rsi_10', 'spread_rsi_10', 'last_10_day_40th_to_50th_pct_price_change_count', 'last_10_day_60th_pct_price_change_count', 'last_10_day_70th_to_80th_pct_volume_change_count', 'last_10_day_80th_to_90th_pct_spread_count', 'last_10_day_40th_pct_spread_count', 'last_10_day_50th_pct_spread_count', 'last_10_day_70th_pct_spread_count', 'last_10_day_80th_pct_spread_count', 'rsi_25', 'volume_rsi_25', 'spread_rsi_25', 'last_25_day_40th_to_50th_pct_price_change_count', 'last_25_day_70th_to_80th_pct_price_change_count', 'last_25_day_80th_to_90th_pct_price_change_count', 'last_25_day_40th_pct_price_change_count', 'last_25_day_50th_pct_price_change_count', 'last_25_day_80th_pct_price_change_count', 'last_25_day_40th_to_50th_pct_volume_change_count', 'last_25_day_50th_to_60th_pct_volume_change_count', 'last_25_day_60th_to_70th_pct_volume_change_count', 'last_25_day_70th_to_80th_pct_volume_change_count', 'last_25_day_80th_to_90th_pct_volume_change_count', 'last_25_day_50th_pct_volume_change_count', 'last_25_day_60th_pct_volume_change_count', 'last_25_day_70th_pct_volume_change_count', 'last_25_day_80th_pct_volume_change_count', 'last_25_day_40th_to_50th_pct_spread_count', 'last_25_day_50th_to_60th_pct_spread_count', 'last_25_day_60th_to_70th_pct_spread_count', 'last_25_day_70th_to_80th_pct_spread_count', 'last_25_day_80th_to_90th_pct_spread_count', 'last_25_day_40th_pct_spread_count', 'last_25_day_60th_pct_spread_count', 'last_25_day_70th_pct_spread_count', 'last_25_day_90th_pct_spread_count', 'rsi_50', 'last_50_day_50th_to_60th_pct_price_change_count', 'last_50_day_60th_to_70th_pct_price_change_count', 'last_50_day_70th_to_80th_pct_price_change_count', 'last_50_day_40th_pct_price_change_count', 'last_50_day_50th_pct_price_change_count', 'last_50_day_70th_pct_price_change_count', 'last_50_day_40th_to_50th_pct_volume_change_count', 'last_50_day_50th_to_60th_pct_volume_change_count', 'last_50_day_50th_pct_volume_change_count', 'last_50_day_80th_pct_volume_change_count', 'last_50_day_40th_to_50th_pct_spread_count', 'last_50_day_50th_to_60th_pct_spread_count', 'last_50_day_60th_to_70th_pct_spread_count', 'last_50_day_70th_to_80th_pct_spread_count', 'last_50_day_80th_to_90th_pct_spread_count', 'last_50_day_40th_pct_spread_count', 'last_50_day_50th_pct_spread_count', 'last_50_day_70th_pct_spread_count', 'last_50_day_90th_pct_spread_count', 'rsi_100', 'last_100_day_60th_to_70th_pct_price_change_count', 'last_100_day_80th_to_90th_pct_price_change_count', 'last_100_day_90th_to_95th_pct_price_change_count', 'last_100_day_40th_pct_price_change_count', 'last_100_day_50th_pct_price_change_count', 'last_100_day_60th_pct_price_change_count', 'last_100_day_80th_pct_price_change_count', 'last_100_day_40th_to_50th_pct_volume_change_count', 'last_100_day_50th_to_60th_pct_volume_change_count', 'last_100_day_60th_to_70th_pct_volume_change_count', 'last_100_day_70th_to_80th_pct_volume_change_count', 'last_100_day_80th_to_90th_pct_volume_change_count', 'last_100_day_40th_pct_volume_change_count', 'last_100_day_60th_pct_volume_change_count', 'last_100_day_80th_pct_volume_change_count', 'last_100_day_90th_pct_volume_change_count', 'last_100_day_40th_to_50th_pct_spread_count', 'last_100_day_50th_to_60th_pct_spread_count', 'last_100_day_60th_to_70th_pct_spread_count', 'last_100_day_70th_to_80th_pct_spread_count', 'last_100_day_90th_to_95th_pct_spread_count', 'last_100_day_40th_pct_spread_count', 'last_100_day_50th_pct_spread_count', 'last_100_day_60th_pct_spread_count', 'last_100_day_80th_pct_spread_count', 'last_100_day_90th_pct_spread_count']

In [91]:
scaler = StandardScaler()
eth_ohlc_scaled = scaler.fit_transform(eth_ohlc[predictors_96])
eth_ohlc_scaled = pd.DataFrame(eth_ohlc_scaled, columns=predictors_96)
eth_ohlc_scaled["tmw_1_0_percent_increase_binary"] = eth_ohlc['tmw_1_0_percent_increase_binary']
eth_ohlc_scaled["tmw_percent_increase_to_avg_high_low"] = eth_ohlc['tmw_percent_increase_to_avg_high_low']

In [92]:
%%capture

# Define hyperparameters to test
units_options = [64, 128, 256]  # Different options for the number of units in Dense layers
dropout_options = [0.3, 0.5, 0.7]  # Different options for dropout rates
learning_rate_options = [0.001, 0.01]  # Different learning rates

# Store results
results = []

for units in units_options:
    for dropout in dropout_options:
        for lr in learning_rate_options:
            model = Sequential([
                Dense(units, activation='relu', input_shape=(96,)),
                Dropout(dropout),
                Dense(units // 2, activation='relu'),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            # Use your function to get predictions and calculate precision
            df = get_backtested_predictions_df_neural(eth_ohlc_scaled, predictors_96, model, "tmw_1_0_percent_increase_binary")
            df['predictions'] = df['probability'] >= 0.55
            score_for_1 = precision_score(df['tmw_1_0_percent_increase_binary'], df['predictions'])
            count_1s = df['predictions'].value_counts().get(1, 0)

            # Store the configuration and score
            results.append({
                'units': units,
                'dropout': dropout,
                'learning_rate': lr,
                'Precision for 1.0%': score_for_1,
                'Count of 1s': count_1s
            })

results_df = pd.DataFrame(results)

In [98]:
sorted_results_df = results_df.sort_values('Precision for 1.0%', ascending=False)
sorted_results_df

,units,dropout,learning_rate,Precision for 1.0%,Count of 1s
8,128,0.5,0.001,0.656522,230
17,256,0.7,0.010,0.648148,216
16,256,0.7,0.001,0.642586,263
11,128,0.7,0.010,0.639175,194
10,128,0.7,0.001,0.630901,233
3,64,0.5,0.010,0.607143,308
14,256,0.5,0.001,0.605422,332
4,64,0.7,0.001,0.604317,139
2,64,0.5,0.001,0.597826,276
9,128,0.5,0.010,0.593939,330


##### The best neural netowrk model is the one with 128 activation nodes in the first layer and 64 in the second layer, with a dropout of 0.5, and learning rate of 0.001. It predicts a 1.0% return 230 times, with a precision score of 0.656522.

### Model 2: Support Vector Machine (SVM)

In [21]:
def get_backtested_predictions_df_svm(data, predictors, model, response_var, start=1000, step=100):
    
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var])
        predictions = model.predict(test[predictors])
        predictions_series = pd.Series(predictions, index=test.index, name="predictions")
        combined = pd.concat([test[response_var], test['tmw_percent_increase_to_avg_high_low'], predictions_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+100)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)
    
    return data_with_predictions

In [28]:
scaler = StandardScaler()
eth_ohlc_scaled = scaler.fit_transform(eth_ohlc[predictors_96])
eth_ohlc_scaled = pd.DataFrame(eth_ohlc_scaled, columns=predictors_96)
eth_ohlc_scaled["tmw_1_0_percent_increase_binary"] = eth_ohlc['tmw_1_0_percent_increase_binary']
eth_ohlc_scaled["tmw_0_0_percent_increase_binary"] = eth_ohlc['tmw_0_0_percent_increase_binary']
eth_ohlc_scaled["tmw_percent_increase_to_avg_high_low"] = eth_ohlc['tmw_percent_increase_to_avg_high_low']

In [72]:
hyper_params = {
    'linear': {'C': [0.01, .02, .03, .04, 0.05, .06, .07, .08, .09, 1, 5, 10]},
    'rbf': {'C': [0.01, .02, .03, .04, 0.05, .06, .07, .08, .09, 1, 5, 10]},
    'poly': {'C': [0.01, .02, .03, .04, 0.05, .06, .07, .08, .09, 1, 5, 10], 'degree': [2,3]},
    'sigmoid': {'C': [0.01, .02, .03, .04, 0.05, .06, .07, .08, .09, 1, 5, 10]}
}

results = []

for kernel, params in hyper_params.items():
    for c in params['C']:
        if kernel == 'poly':
            for degree in params.get('degree', []):
                model = SVC(kernel=kernel, C=c, degree=degree)
                df = get_backtested_predictions_df_svm(eth_ohlc_scaled, predictors_96, model, "tmw_1_0_percent_increase_binary")
                score_for_1 = precision_score(df['tmw_1_0_percent_increase_binary'], df['predictions'], pos_label=1)
                count_1s = df['predictions'].value_counts().get(1, 0)
                results.append({
                    'Kernel': kernel,
                    'C': c,
                    'Degree': degree,
                    'Count of 1s': count_1s,
                    'Precision for 1.0%': score_for_1
                })
        else:
            model = SVC(kernel=kernel, C=c)
            df = get_backtested_predictions_df_svm(eth_ohlc_scaled, predictors_96, model, "tmw_1_0_percent_increase_binary")
            score_for_1 = precision_score(df['tmw_1_0_percent_increase_binary'], df['predictions'])
            count_1s = df['predictions'].value_counts().get(1, 0)
            results.append({
                'Kernel': kernel,
                'C': c,
                'Degree': None,
                'Count of 1s': count_1s,
                'Precision for 1.0%': score_for_1,
            })

# Convert results to DataFrame for better visualization and analysis
results_df = pd.DataFrame(results)

In [79]:
# Assuming results_df is already defined and contains the relevant data
sorted_results_df = results_df.sort_values('Precision for 1.0%', ascending=False)
sorted_results_df

,Kernel,C,Degree,Count of 1s,Precision for 1.0%
25,poly,0.01,3.0,1,1.000000
27,poly,0.02,3.0,1,1.000000
30,poly,0.04,2.0,6,0.666667
32,poly,0.05,2.0,19,0.631579
34,poly,0.06,2.0,39,0.589744
51,sigmoid,0.04,NaN,583,0.583190
52,sigmoid,0.05,NaN,612,0.581699
39,poly,0.08,3.0,152,0.578947
49,sigmoid,0.02,NaN,479,0.578288
53,sigmoid,0.06,NaN,630,0.577778


##### The best SVM model is the one using the sigmoid kernel with cost budget of 0.05. It predicts a 1.0% return 612 times, with a precision score of 0.581699.

### Model 3: Catboost

In [125]:
def get_backtested_predictions_df(data, predictors, model, response_var, start=1000, step=100):
    
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var])
        probability = model.predict_proba(test[predictors])[:,1]
        proba_series = pd.Series(probability, index=test.index, name="probability")
        combined = pd.concat([test[response_var], test['tmw_percent_increase_to_avg_high_low'], proba_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+100)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)
    
    return data_with_predictions

In [ ]:
%%capture

# Define hyperparameters to test
iterations_options = [200, 300, 400]
learning_rate_options = [0.1, 0.2, 0.3, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019]
depth_options = [5, 6, 7]

# Store results
results = []

for iterations in iterations_options:
    for lr in learning_rate_options:
        for depth in depth_options:
            model = cb.CatBoostClassifier(
                iterations=iterations,
                learning_rate=lr,
                depth=depth,
                verbose=False,
                random_seed=1
            )

            # Use your function to get predictions and calculate precision
            df = get_backtested_predictions_df(eth_ohlc, predictors_96, model, "tmw_1_0_percent_increase_binary")
            df['predictions'] = df['probability'] >= 0.55
            score_for_1 = precision_score(df['tmw_1_0_percent_increase_binary'], df['predictions'])
            count_1s = df['predictions'].value_counts().get(1, 0)

            # Store the configuration and score
            results.append({
                'iterations': iterations,
                'learning_rate': lr,
                'depth': depth,
                'Count of 1s': count_1s,
                'Precision for 1.0%': score_for_1
            })

            print(iterations, lr, depth, count_1s, score_for_1)

results_df = pd.DataFrame(results)


In [134]:
sorted_results_df = results_df.sort_values('Precision for 1.0%', ascending=False)
sorted_results_df

,iterations,learning_rate,depth,Count of 1s,Precision for 1.0%
0,300,0.019,6,415,0.619277
9,200,0.010,6,332,0.614458
1,300,0.015,6,412,0.614078
2,300,0.016,6,391,0.611253
3,300,0.013,6,405,0.607407
4,300,0.014,6,406,0.603448
5,300,0.018,6,443,0.602709
10,300,0.010,6,369,0.601626
11,300,0.020,6,439,0.601367
12,200,0.010,5,320,0.600000


##### The best catboost model is the one with 300 iterations, learning rate of 0.019, and depth of 6. It predicts a 1.0% return 415 times, with a precision score of 0.619277.

### Model 4: Random Forest Classifier

In [ ]:
# Define hyperparameters to test
n_estimators_options = [200, 400] # number of trees in the forest
min_samples_split_options = [75, 100, 125, 150, 175]

# Store results
results = []

for n_estimators in n_estimators_options:
    for min_samples_split in min_samples_split_options:
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            min_samples_split=min_samples_split,
            random_state=1
        )

        # Use your function to get predictions and calculate precision
        df = get_backtested_predictions_df(eth_ohlc, predictors_96, model, "tmw_1_0_percent_increase_binary")
        df['predictions'] = df['probability'] >= 0.55
        score_for_1 = precision_score(df['tmw_1_0_percent_increase_binary'], df['predictions'])
        count_1s = df['predictions'].value_counts().get(1, 0)

        # Store the configuration and score
        results.append({
            'n_estimators': n_estimators,
            'min_samples_split': min_samples_split,
            'Count of 1s': count_1s,
            'Precision for 1.0%': score_for_1
        })

        print(n_estimators, min_samples_split, count_1s, score_for_1)

results_df = pd.DataFrame(results)


In [137]:
sorted_results_df = results_df.sort_values('Precision for 1.0%', ascending=False)
sorted_results_df

,n_estimators,min_samples_split,Count of 1s,Precision for 1.0%
3,200,150,352,0.590909
4,200,175,349,0.581662
5,400,75,381,0.580052
6,400,100,374,0.577540
9,400,175,355,0.577465
8,400,150,354,0.576271
0,200,75,375,0.576000
7,400,125,353,0.575071
1,200,100,374,0.574866
2,200,125,340,0.570588


##### The best random forest model is the one with 200 bootstraps and learning rate of 150 min_samples_split. It predicts a 1.0% return 352 times, with a precision score of 0.590909

### LightGBM

In [139]:
%%capture

num_leaves_values = [15, 31, 45]
max_depth_values = [5, 6, 7]
learning_rate_values = [0.05, 0.01, 0.015]

# Store results
results = []

for num_leaves in num_leaves_values:
    for max_depth in max_depth_values:
        for learning_rate in learning_rate_values:

            model = lgb.LGBMClassifier(num_leaves=num_leaves, 
                                        max_depth=max_depth, 
                                        learning_rate=learning_rate, 
                                        random_state=1)

            # Use your function to get predictions and calculate precision
            df = get_backtested_predictions_df(eth_ohlc, predictors_96, model, "tmw_1_0_percent_increase_binary")
            df['predictions'] = df['probability'] >= 0.55
            score_for_1 = precision_score(df['tmw_1_0_percent_increase_binary'], df['predictions'])
            count_1s = df['predictions'].value_counts().get(1, 0)

            # Store the configuration and score
            results.append({
                'num_leaves': num_leaves,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'Count of 1s': count_1s,
                'Precision for 1.0%': score_for_1
            })

            print(num_leaves, max_depth, learning_rate, count_1s, score_for_1)

results_df = pd.DataFrame(results)

In [140]:
sorted_results_df = results_df.sort_values('Precision for 1.0%', ascending=False)
sorted_results_df

,num_leaves,max_depth,learning_rate,Count of 1s,Precision for 1.0%
1,15,5,0.010,354,0.584746
7,15,7,0.010,368,0.578804
20,45,5,0.015,465,0.578495
11,31,5,0.015,465,0.578495
16,31,7,0.010,410,0.578049
8,15,7,0.015,459,0.577342
5,15,6,0.015,472,0.576271
25,45,7,0.010,439,0.571754
22,45,6,0.010,414,0.570048
19,45,5,0.010,363,0.567493


##### The best LightGBM model is the one with 200 num_leaves, max_depth, and learning_rate. It predicts a 1.0% return 354 times, with a precision score of 0.584746